In [25]:
from pathlib import Path
import requests
from tqdm import tqdm

phoible_data_url = "https://raw.githubusercontent.com/phoible/dev/v2.0/data/phoible.csv"
phobile_file_path = "./data/phoible.csv"

if not Path(phobile_file_path).exists():
    response = requests.get(phoible_data_url, stream=True)

    with open(phobile_file_path, "wb") as fh:
        for data in tqdm(response.iter_content()):
            fh.write(data)

In [26]:
import pandas as pd

df = pd.read_csv(phobile_file_path)
df[df["LanguageName"] == "Welsh"].head(5)

/var/folders/w1/813808f13m14dv7gd5ystxbc0000gn/T/ipykernel_26583/4029157069.py:3: DtypeWarning: Columns (4,7,8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(phobile_file_path)


,InventoryID,Glottocode,ISO6393,LanguageName,SpecificDialect,GlyphID,Phoneme,Allophones,Marginal,SegmentClass,...,retractedTongueRoot,advancedTongueRoot,periodicGlottalSource,epilaryngealSource,spreadGlottis,constrictedGlottis,fortis,raisedLarynxEjective,loweredLarynxImplosive,click
86146,2406,wels1247,cym,Welsh,Welsh (Llanwrtyd),0061,a,NaN,False,vowel,...,-,-,+,-,-,-,0,-,-,0
86147,2406,wels1247,cym,Welsh,Welsh (Llanwrtyd),0061+0069,ai,NaN,False,vowel,...,-,-,+,-,-,-,0,-,-,0
86148,2406,wels1247,cym,Welsh,Welsh (Llanwrtyd),0061+0075,au,NaN,False,vowel,...,-,-,+,-,-,-,0,-,-,0
86149,2406,wels1247,cym,Welsh,Welsh (Llanwrtyd),0251+02D0,ɑː,NaN,False,vowel,...,-,-,+,-,-,-,0,-,-,0
86150,2406,wels1247,cym,Welsh,Welsh (Llanwrtyd),00F0,ð,NaN,False,consonant,...,0,0,+,-,-,-,-,-,-,-


In [27]:
consonants_plumonic_df = pd.read_csv("./consonants_plumonic.csv", dtype=str, index_col=[0], keep_default_na=False, na_values=["-1"])
consonants_plumonic_df

,Bilabial,Bilabial_v,Labiodental,Labiodental_v,Dental,Dental_v,Alveolar,Alveolar_v,Postalveolar,Postalveolar_v,...,Palatal,Palatal_v,Velar,Velar_v,Uvular,Uvular_v,Pharyngeal,Pharyngeal_v,Glottal,Glottal_v
Plosive,p,b,,,t̪,d̪,t,d,,,...,c,ɟ,k,ɡ,q,ɢ,,NaN,ʔ,NaN
Nasal,,m,,ɱ,,n̪,,n,,,...,,ɲ,,ŋ,,ɴ,NaN,NaN,NaN,NaN
Trill,,ʙ,,,,r̪,,r,,,...,,,NaN,NaN,,ʀ,,,NaN,NaN
Tap or Flap,,,,ⱱ,,ɾ̪,,ɾ,,,...,,,NaN,NaN,,,,,NaN,NaN
Fricative,ɸ,β,f,v,θ,ð,s,z,ʃ,ʒ,...,ç,ʝ,x,ɣ,χ,ʁ,ħ,ʕ,h,ɦ
Lateral fricative,NaN,NaN,NaN,NaN,,,ɬ,ɮ,,,...,,,,,,,NaN,NaN,NaN,NaN
Approximant,,,,ʋ,,,,ɹ,,,...,,j,,ɰ,,,,,NaN,NaN
Lateral approximan,NaN,NaN,NaN,NaN,,l̪,,l,,,...,,ʎ,,ʟ,,,NaN,NaN,NaN,NaN
